In [1]:
import matplotlib.pyplot as plt
import subprocess
import re

# pub / sub Throughput

In [2]:
throughput_repeat = 10000
n_pub = 1
n_sub = 1
n_channel = 4

In [3]:
#CLOA
batcmd=f"ssh ec2-user@52.3.187.162 -i ~/cloakey.pem ./bench 172.31.21.234 4242 {throughput_repeat} {n_pub} {n_sub} {n_channel}"
result = subprocess.check_output(batcmd, shell=True, text = True)
result = result.strip()

result_ls = result.split('rps')
pub_throughput = result_ls[0].split(' ')[-1]
sub_throughput = result_ls[1].split(' ')[-1]
cloa_throughput = list(map(int , [pub_throughput, sub_throughput]))

In [4]:
# Redis
batcmd=f"ssh ec2-user@52.3.187.162 -i ~/cloakey.pem ./bench 172.31.21.234 6379 {throughput_repeat} {n_pub} {n_sub} {n_channel} redis"
result = subprocess.check_output(batcmd, shell=True, text = True)
result = result.strip()

result_ls = result.split('rps')
pub_throughput = result_ls[0].split(' ')[-1]
sub_throughput = result_ls[1].split(' ')[-1]
redis_throughput = list(map(int,[pub_throughput, sub_throughput]))

In [5]:
# Redis-C
batcmd=f"ssh ec2-user@52.3.187.162 -i ~/cloakey.pem ./bench 172.31.21.234 5000 {throughput_repeat} {n_pub} {n_sub} {n_channel} redis"
result = subprocess.check_output(batcmd, shell=True, text = True)
result = result.strip()

result_ls = result.split('rps')
pub_throughput = result_ls[0].split(' ')[-1]
sub_throughput = result_ls[1].split(' ')[-1]
redis_c_throughput = list(map(int,[pub_throughput, sub_throughput]))

## Get / Set Throughput

In [6]:
# Redis SET
batcmd=f"ssh ec2-user@52.3.187.162 -i ~/cloakey.pem ./redis-benchmark -h 172.31.21.234 -t set"
result = subprocess.check_output(batcmd, shell=True, text = True)
result = result.strip()
result

'SET: -nan\nSET: 82368.00\nSET: 82908.00\nSET: 83036.00\nSET: 83101.00\n====== SET ======\n  100000 requests completed in 1.21 seconds\n  50 parallel clients\n  3 bytes payload\n  keep alive: 1\n\n99.92% <= 1 milliseconds\n100.00% <= 1 milliseconds\n82987.55 requests per second'

In [7]:
p = re.compile(r'(\d+\.\d+) requests per second')

m = p.search(result)
redis_set_throughput = [float(m.group(1))]

In [8]:
# Redis-c SET
batcmd=f"ssh ec2-user@52.3.187.162 -i ~/cloakey.pem ./redis-benchmark -h 172.31.21.234 -p 5000 -t set"
result = subprocess.check_output(batcmd, shell=True, text = True)
result = result.strip()
result

'SET: -nan\nSET: 84388.00\nSET: 84488.00\nSET: 84588.00\nSET: 84641.00\n====== SET ======\n  100000 requests completed in 1.18 seconds\n  50 parallel clients\n  3 bytes payload\n  keep alive: 1\n\n99.96% <= 1 milliseconds\n99.99% <= 2 milliseconds\n99.99% <= 3 milliseconds\n100.00% <= 4 milliseconds\n100.00% <= 4 milliseconds\n84674.01 requests per second'

In [9]:
p = re.compile(r'(\d+\.\d+) requests per second')

m = p.search(result)
redis_c_set_throughput = [float(m.group(1))]

In [10]:
# Redis GET
batcmd=f"ssh ec2-user@52.3.187.162 -i ~/cloakey.pem ./redis-benchmark -h 172.31.21.234 -t get"
result = subprocess.check_output(batcmd, shell=True, text = True)
result = result.strip()
result

'GET: -nan\nGET: 83856.00\nGET: 84316.00\nGET: 84474.66\nGET: 84571.00\n====== GET ======\n  100000 requests completed in 1.18 seconds\n  50 parallel clients\n  3 bytes payload\n  keep alive: 1\n\n99.95% <= 1 milliseconds\n99.97% <= 3 milliseconds\n99.98% <= 4 milliseconds\n100.00% <= 4 milliseconds\n84674.01 requests per second'

In [11]:
p = re.compile(r'(\d+\.\d+) requests per second')

m = p.search(result)
redis_get_throughput = [float(m.group(1))]

In [12]:
# Redis-c GET
batcmd=f"ssh ec2-user@52.3.187.162 -i ~/cloakey.pem ./redis-benchmark -h 172.31.21.234 -p 5000 -t get"
result = subprocess.check_output(batcmd, shell=True, text = True)
result = result.strip()
result

'GET: 0.00\nGET: 83386.46\nGET: 83678.65\nGET: 83917.45\nGET: 84052.95\n====== GET ======\n  100000 requests completed in 1.19 seconds\n  50 parallel clients\n  3 bytes payload\n  keep alive: 1\n\n99.94% <= 1 milliseconds\n100.00% <= 1 milliseconds\n84033.61 requests per second'

In [13]:
p = re.compile(r'(\d+\.\d+) requests per second')

m = p.search(result)
redis_c_get_throughput = [float(m.group(1))]

### 패턴 만들때 유의할점

#틀린 패턴 : 모든 부분을 다 설명할 필요없다
p = re.compile(r'.*(\d+\.[0-9]*) requests per second')

#올바른 패턴 : 찾을 부분만 설명한다
p = re.compile(r'(\d+\.\d+) requests per second')

m = p.search(result)
m.group(1)

# E2E Latency

In [17]:
REPEAT = 10000

In [18]:

p = re.compile('avg Latency ([0-9|.]*)')

In [19]:
#redis
batcmd = f"ssh ec2-user@18.235.181.217 -i ~/cloakey.pem ./e2e -h 172.31.21.234:6379 -b redis -c {REPEAT}"
result = subprocess.check_output(batcmd, shell=True, text = True)
result

m = p.search(result)
cloa_e2e_latency = [float(m.group(1))]

In [ ]:
#cloa
batcmd = f"ssh ec2-user@18.235.181.217 -i ~/cloakey.pem ./e2e -h 172.31.21.234:4242 -b cloa -c {REPEAT}"
result = subprocess.check_output(batcmd, shell=True, text = True)
result

m = p.search(result)

redis_e2e_latency = [float(m.group(1))]

In [ ]:
#redis-c
batcmd = f"ssh ec2-user@18.235.181.217 -i ~/cloakey.pem ./e2e -h 172.31.21.234:6379 -b redis -c {REPEAT}"
result = subprocess.check_output(batcmd, shell=True, text = True)
result

m = p.search(result)
redis_c_e2e_latency = [float(m.group(1))]

In [ ]:
%matplotlib notebook

In [ ]:
plt.scatter(redis_e2e_latency[0],redis_throughput[0], label = "redis_pub",  color= "red", alpha = 0.5, marker = 'v')
plt.scatter(redis_e2e_latency[0],redis_throughput[1], label = "redis_sub", color= "red", alpha = 0.5, marker = '^')

plt.scatter(redis_c_e2e_latency[0],redis_c_throughput[0], label = "redis_c_pub",  color= "yellow", alpha = 0.5, marker = 'v')
plt.scatter(redis_c_e2e_latency[0],redis_c_throughput[1], label = "redis_c_sub", color= "yellow", alpha = 0.5, marker = '^')

plt.scatter(cloa_e2e_latency[0],cloa_throughput[0], label = "cloa_pub", color = "blue", alpha = 0.5, marker = 'v')
plt.scatter(cloa_e2e_latency[0],cloa_throughput[1], label = "cloa_sub", color = "blue", alpha = 0.5, marker = '^')


plt.legend()
plt.xlabel("EndToEnd Latency (ms)")
plt.ylabel("Throughput (rps)")
plt.title("Latency-Throughput Graph")
plt.tight_layout()
plt.show()